# TFM
### Ismael Franco Hernando

# Imports

In [1]:
import sys, os, cv2, glob, pickle
import scipy.ndimage
import plotly.express as px
import numpy as np
import pandas as pd
from skimage import io
import seaborn as sns
import scipy as sp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from PIL import Image
from statistics import mean
import matplotlib.image
import matplotlib.pyplot
from tqdm import tqdm
from mysql.connector import (connection)
from jupyter_plotly_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Conexión y pruebas en la base de datos

Nos conectamos a la base de datos a través del usuario, contraseña, host y nombre de la base de datos. En caso de que haya algún problema devolverá un error.

In [45]:
try:
    cnx = connection.MySQLConnection(user='leer', 
                                     password='candanal',
                                     host='138.100.82.178',
                                     database='deepquality')

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Usuario o contraseña incorrecto")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("La base de datos no existe")
    else:
        print(err)

In [3]:
# Contamos el número de registros que hay en la tbala correspondiente a las imágenes 1D
q01 = "SELECT count(*) FROM GR_LPValues"
r01 = pd.read_sql(q01,cnx)
r01

,count(*)
0,115972


In [4]:
# Contamos el número de registros que hay en la tbala correspondiente a las imágenes 2D
q02 = "SELECT count(*) FROM GR_QPValues"
r02 = pd.read_sql(q02,cnx)
r02

,count(*)
0,1272312


In [5]:
# Contamos el número de sensores que se usan para tomar los datos
q03 = "SELECT count(*) FROM Gauges"
r03 = pd.read_sql(q03,cnx)
r03

,count(*)
0,6


In [6]:
# Contamos el número de bobinas que se han evaluado
q04 = "SELECT count(*) FROM V_Coils"
r04 = pd.read_sql(q04,cnx)
r04

,count(*)
0,1160


# Bobinas clasificadas

In [7]:
# Guardamos y mostramos las bobinas clasificadas
q05 = "SELECT * FROM V_Coils"
clasif = pd.read_sql(q05,cnx)
clasif

,id,SID,PLANT,NAME,STARTTIME,MATERIAL,LENGTH,WIDTH,THICK,WEIGHT,BASEPID,CLASSLABEL,Label,ZnMin,ZnMax
0,1161,226031456,1066.0,226031456,2022-01-28,0.0,792.0,1183.0,3.0,22.0,220100.0,NOK,0,60,80
1,1162,226101621,1066.0,226101621,2022-01-31,0.0,273.0,1252.0,4.0,10.0,220100.0,NOK,2,215,258
2,1163,226120037,1066.0,226120037,2022-01-31,0.0,599.0,1305.0,2.0,15.0,220100.0,OK,1,155,175
3,1164,225387481,1066.0,225387481,2022-01-13,0.0,268.0,1502.0,5.0,15.0,220100.0,OK,1,60,80
4,1165,225406471,1066.0,225406471,2022-01-13,0.0,735.0,1158.0,2.0,19.0,220100.0,NOK,0,45,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155,2316,233642421,1066.0,233642421,2022-08-31,0.0,576.0,1422.0,3.0,24.0,220800.0,OK,2,138,158
1156,2317,233445180,1066.0,233445180,2022-08-25,0.0,513.0,1437.0,4.0,23.0,220800.0,OK,1,125,145
1157,2318,232624927,1066.0,232624927,2022-08-01,0.0,345.0,1482.0,2.0,11.0,220800.0,OK,2,155,175
1158,2319,232635130,1066.0,232635130,2022-08-01,0.0,457.0,968.0,3.0,11.0,220800.0,OK,1,88,108


# Imágenes 1D


In [8]:
# Guardamos y mostramos los valores correspondientes a las imágenes 1D
q06 = "SELECT * FROM GR_LPValues"
unoD = pd.read_sql(q06,cnx)
unoD

,COILID,TILEID,MID,PLANT,SIDE,MEAN,MAX,MIN,COUNT
0,228318255,116480.0,124.0,1066.0,T,143.32179,143.32179,143.32179,1.0
1,228318255,116480.0,201.0,1066.0,T,145.10339,145.10339,145.10339,1.0
2,228318255,116480.0,202.0,1066.0,T,138.98370,138.98370,138.98370,1.0
3,228318255,116480.0,123.0,1066.0,T,145.78580,145.78580,145.78580,1.0
4,228318255,119040.0,124.0,1066.0,T,148.77290,148.77290,148.77290,1.0
...,...,...,...,...,...,...,...,...,...
115967,232800547,0.0,123.0,1066.0,T,207.00020,207.00020,207.00020,1.0
115968,232800547,5120.0,124.0,1066.0,T,89.30820,89.30820,89.30820,1.0
115969,232800547,5120.0,201.0,1066.0,T,248.35600,248.35600,248.35600,1.0
115970,232800547,5120.0,202.0,1066.0,T,76.74000,76.74000,76.74000,1.0


In [18]:
# Ordenamos el DF a través de la 1D de la bobina y de las diferentes tejas que tiene
unoD = unoD.sort_values(['COILID', 'TILEID']).reset_index(drop=True)

In [33]:
# Función que recibe un DF de datos sin clasificar y el DF con las bobinas clasificadas. Evaluará teja a teja 
# y devolverá el número de fallos totales en la bobina, además a cada teja le colocará una label según si se 
# encuentra en correscto estado, OK, y otra en caso contrario, NOK.
def analizador(datos, clasificados):
    # Creamos el DF que se devolverá con las diferentes columnas
    rDF = pd.DataFrame(columns = ['COILID' , 'TILEID', 'MID', 'MEAN' , 'ZnMax', 'ZnMin', 
                                  'datosMax', 'datosMin', 'CLASSLABEL', 'LabelVerificada', 'Fallos'])
    
    # Inicializamos los parámetros con los que se trabajará a lo largo de la función
    sid = -1 # Contiene la ID de la bobina evaluada
    znMax = -1 # Valor máximo de ZN aceptado en la bobina
    znMin = -1 # Valor mínimo de ZN aceptado en la bobina
    label = 'null' # Contendrá la label según si cumple o no los requisitos
    fallosTotales = 0 # Contiene el número de fallos totales en una bobina
    fallosAntes = 0 # Contiene el número de fallos totales en una bobina en la anterior iteración
    tileidAnterior = -1 # Contiene la teja evaluada en la anterior iteraccioón
    
    # Se recorren todas las tejas de todas las bobinas para evaluarlas
    for i in range(len(datos)):
        #Obtenemos la ID de la bobina a la que pertenece la teja a evalaur
        id = datos['COILID'][i]
        
        # En caso de que la ID sea de una nueva bobina se cargan los nuevos datos
        if sid != id:
            # Se reinician los contadores de fallos
            fallosTotales = 0
            fallosAntes = 0
            # Se obtiene el registro de la bobina evaluada
            registro = clasificados.loc[clasificados['SID']==id]
            # Se guardan su ID, valores max y min de ZN y su label
            sid = registro['SID'].iloc[0]
            znMax = registro['ZnMax'].iloc[0]
            znMin = registro['ZnMin'].iloc[0]
            label = registro['CLASSLABEL'].iloc[0]
        
        # Obtenemoss el ZN max y min que se ha medido en la teja
        datosMax = datos['MAX'][i]
        datosMin = datos['MIN'][i]
        # Obtenemos la ID de la teja a evaluar
        tileid = datos['TILEID'][i]
        # En caso de que sea una teja distinta a la anterior iteracción, se coloca a Falso el 
        # haber encontrado un error 
        if tileid != tileidAnterior:
            errorEncontrado = False
            
        # Obtenemos el sensor que ha medido los datos
        mid = datos['MID'][i]
        # Obtenemos el valor medio de ZN en la teja
        mean = datos['MEAN'][i]
        
        # Evaluamos si cumple o no con los valores máximos y mínimos de ZN. Además, no se tiene en cuenta
        # el reinicio del medidor del sensor donde los valores de ZN valen 0
        verificado = 'OK'
        if (round(datosMax,1) > znMax or round(datosMin,1) < znMin) and (datosMax > 0 and datosMin > 0):
            # En caso de no cumplir con los requisitos se le evalua como NOK
            verificado = 'NOK'
            # Si en la teja evaluada anteriormente ya había un error no se aumenta el contador,
            # en caso contrario si que se actualzia
            if errorEncontrado == False:
                fallosTotales = fallosTotales + 1
                errorEncontrado = True
        
        # Añadimos al DF a devolver los valores tras su evaluación
        rDF.loc[i] = [id, tileid, mid, mean, znMax, znMin, datosMax, datosMin, label, verificado, fallosTotales]
        
        # Si al evaluar la teja se encuentra un fallo se actualizan los errores de la bobina en el DF
        if fallosTotales > fallosAntes:
            fallosAntes = fallosTotales
            rDF.loc[rDF.COILID==id, 'Fallos'] = fallosTotales
        
        # Se almacena la ID de la teja anterior
        tileidAnterior = tileid 
    
    # Se devuelve el DF con todas las bobinas y tejas evaluadas
    return rDF
            
            

In [34]:
# Se analizan todos los datos 1D
aDF = analizador(unoD, clasif)
aDF

,COILID,TILEID,MID,MEAN,ZnMax,ZnMin,datosMax,datosMin,CLASSLABEL,LabelVerificada,Fallos
0,225216688,0.0,123.0,88.94490,158,138,88.94490,88.94490,OK,NOK,26
1,225216688,0.0,202.0,160.64000,158,138,160.64000,160.64000,OK,NOK,26
2,225216688,0.0,201.0,110.00800,158,138,110.00800,110.00800,OK,NOK,26
3,225216688,0.0,124.0,182.08980,158,138,182.08980,182.08980,OK,NOK,26
4,225216688,5120.0,123.0,81.65457,158,138,81.65457,81.65457,OK,NOK,26
...,...,...,...,...,...,...,...,...,...,...,...
115967,233667056,124672.0,123.0,62.79973,70,50,62.79973,62.79973,OK,OK,15
115968,233667056,129792.0,124.0,41.93915,70,50,41.93915,41.93915,OK,NOK,15
115969,233667056,129792.0,202.0,43.21667,70,50,43.21667,43.21667,OK,NOK,15
115970,233667056,129792.0,123.0,65.04555,70,50,65.04555,65.04555,OK,OK,15


In [35]:
#Serialización del DF de los datos 1D
aDF.to_pickle("./serializados/unoD.pkl") 

In [42]:
# Se muestran la evaluación de los primeros 60 registros de la bobina con ID 233667056
aDF[aDF['COILID']=='233667056'].head(60)

,COILID,TILEID,MID,MEAN,ZnMax,ZnMin,datosMax,datosMin,CLASSLABEL,LabelVerificada,Fallos
115868,233667056,0.0,123.0,58.29216,70,50,58.29216,58.29216,OK,OK,15
115869,233667056,0.0,202.0,58.14000,70,50,58.14000,58.14000,OK,OK,15
115870,233667056,0.0,201.0,55.21200,70,50,55.21200,55.21200,OK,OK,15
115871,233667056,0.0,124.0,56.54638,70,50,56.54638,56.54638,OK,OK,15
115872,233667056,5120.0,124.0,58.22531,70,50,58.22531,58.22531,OK,OK,15
115873,233667056,5120.0,201.0,53.76493,70,50,53.76493,53.76493,OK,OK,15
115874,233667056,5120.0,202.0,59.84417,70,50,59.84417,59.84417,OK,OK,15
115875,233667056,5120.0,123.0,58.92805,70,50,58.92805,58.92805,OK,OK,15
115876,233667056,10240.0,124.0,56.82838,70,50,56.82838,56.82838,OK,OK,15
115877,233667056,10240.0,201.0,53.22169,70,50,53.22169,53.22169,OK,OK,15


In [62]:
# Se muestran las estadísticas obtenidos por parte de los expertos y por la función generada
print("Los valores de los expertos han sido:\n",aDF['CLASSLABEL'].value_counts())
print("\nLos valores que se han obtenido son:\n",aDF['LabelVerificada'].value_counts().sort_values())

Los valores de los expertos han sido:
 OK     85465
NOK    30507
Name: CLASSLABEL, dtype: int64

Los valores que se han obtenido son:
 OK     51221
NOK    64751
Name: LabelVerificada, dtype: int64


# Imágenes 2D

In [46]:
q07 = "SELECT * FROM GR_QPValues"
dosD = pd.read_sql(q07,cnx)
dosD

,COILID,TILEID,MID,PLANT,SIDE,MEAN,MAX,MIN,COUNT
0,227501982,123817.0,1243.0,1066.0,T,126.82590,126.82590,126.82590,1.0
1,227501982,123817.0,1234.0,1066.0,T,184.95621,184.95621,184.95621,1.0
2,227501982,125353.0,1243.0,1066.0,T,125.99920,125.99920,125.99920,1.0
3,227501982,125353.0,1234.0,1066.0,T,183.72990,183.72990,183.72990,1.0
4,227501982,126889.0,1243.0,1066.0,T,134.66490,134.66490,134.66490,1.0
...,...,...,...,...,...,...,...,...,...
1272307,231628391,56.0,1243.0,1066.0,T,299.99670,299.99670,299.99670,1.0
1272308,231628391,1592.0,1243.0,1066.0,T,299.99670,299.99670,299.99670,1.0
1272309,231628391,3384.0,1243.0,1066.0,T,299.08820,299.08820,299.08820,1.0
1272310,231628391,5176.0,1243.0,1066.0,T,290.91199,290.91199,290.91199,1.0


In [48]:
dosD = dosD.sort_values(['COILID', 'TILEID']).reset_index(drop=True)
dosD

,COILID,TILEID,MID,PLANT,SIDE,MEAN,MAX,MIN,COUNT
0,225216688,0.0,1243.0,1066.0,T,139.32570,139.32570,139.32570,1.0
1,225216688,0.0,1234.0,1066.0,T,147.09500,147.09500,147.09500,1.0
2,225216688,28.0,1243.0,1066.0,T,138.24091,138.24091,138.24091,1.0
3,225216688,28.0,1234.0,1066.0,T,143.53551,143.53551,143.53551,1.0
4,225216688,56.0,1243.0,1066.0,T,137.70560,137.70560,137.70560,1.0
...,...,...,...,...,...,...,...,...,...
1272307,233667056,130986.0,1243.0,1066.0,T,44.40857,44.40857,44.40857,1.0
1272308,233667056,131015.0,1234.0,1066.0,T,63.27512,63.27512,63.27512,1.0
1272309,233667056,131015.0,1243.0,1066.0,T,44.03534,44.03534,44.03534,1.0
1272310,233667056,131043.0,1243.0,1066.0,T,52.01275,52.01275,52.01275,1.0


In [63]:
aDF2 = analizador(dosD, clasif)
aDF2

,COILID,TILEID,MID,MEAN,ZnMax,ZnMin,datosMax,datosMin,CLASSLABEL,LabelVerificada,Fallos
1,227501982,123817.0,1234.0,184.95621,175,155,184.95621,184.95621,OK,NOK,5
3,227501982,125353.0,1234.0,183.72990,175,155,183.72990,183.72990,OK,NOK,5
0,227501982,123817.0,1243.0,126.82590,175,155,126.82590,126.82590,OK,NOK,5
2,227501982,125353.0,1243.0,125.99920,175,155,125.99920,125.99920,OK,NOK,5
4,227501982,126889.0,1243.0,134.66490,175,155,134.66490,134.66490,OK,NOK,5
18,227522012,115541.0,1234.0,76.45282,80,60,76.45282,76.45282,OK,OK,26
36,227522012,115598.0,1234.0,71.38795,80,60,71.38795,71.38795,OK,OK,26
20,227522012,117333.0,1234.0,78.13152,80,60,78.13152,78.13152,OK,OK,26
34,227522012,117390.0,1234.0,71.59948,80,60,71.59948,71.59948,OK,OK,26
6,227522012,119352.0,1234.0,80.72434,80,60,80.72434,80.72434,OK,NOK,26
